# CareerForge AI - Backend Playground
This notebook implements the agent logic using `google-adk`.

In [ ]:
import os
import asyncio
from dotenv import load_dotenv
from google.adk import Agent, Runner
from google.adk.models import Gemini
from google.adk.sessions import InMemorySessionService

# Load from parent directory .env
load_dotenv(dotenv_path="../.env")
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    print("WARNING: GOOGLE_API_KEY not found!")
else:
    print("API Key loaded.")

In [ ]:
# Initialize Common Services
if api_key:
    model = Gemini(model="gemini-1.5-pro", api_key=api_key)
    session_service = InMemorySessionService()
    print("Services initialized.")

In [ ]:
# Define Agents

mentor_prompt = """
You are the 'Mentor' in CareerForge AI. You are a friendly, patient, and analogy-loving professor.
Your goal is to explain technical concepts to a student.
Guidelines:
1. Use a real-world analogy (not computer related) to explain the concept first.
2. Then explain the technical details.
3. Keep it concise (under 200 words).
4. End with a question to check understanding.
"""

manager_prompt = """
You are the 'Manager' in CareerForge AI. You are a busy, direct, but fair CTO.
Your goal is to assign a realistic work task to an intern based on what they just learned.
Output Format:
Subject: [Email Subject]
Body: [Email Body explaining the business problem and what needs to be done. Be realistic, mention 'clients' or 'deadlines'.]
Task: [Specific coding instructions]
"""

reviewer_prompt = """
You are the 'Reviewer' in CareerForge AI. You are a senior engineer who is strict about code quality, security, and best practices.
Analyze the code.
1. Does it solve the task?
2. Are there security issues?
3. Is the style correct?
If it's good, say 'APPROVED'.
If it's bad, say 'CHANGES REQUESTED' and explain why, citing specific lines.
"""

if api_key:
    mentor_agent = Agent(model=model, name="Mentor", instruction=mentor_prompt)
    manager_agent = Agent(model=model, name="Manager", instruction=manager_prompt)
    reviewer_agent = Agent(model=model, name="Reviewer", instruction=reviewer_prompt)

    orchestrator_prompt = """
    You are the Orchestrator. Route the user to the right agent:
    - Learning/Questions -> MENTOR
    - Asking for Task -> MANAGER
    - Submitting Code -> REVIEWER
    Return ONLY the agent name.
    """
    orchestrator_agent = Agent(model=model, name="Orchestrator", instruction=orchestrator_prompt)
    print("Agents defined.")

In [ ]:
# Test Run Helper
async def run_agent(agent_name, user_input):
    if not api_key: return
    
    target_agent = None
    if agent_name == "MENTOR": target_agent = mentor_agent
    elif agent_name == "MANAGER": target_agent = manager_agent
    elif agent_name == "REVIEWER": target_agent = reviewer_agent
    else: 
        print(f"Unknown agent: {agent_name}")
        return

    runner = Runner(agent=target_agent, session_service=session_service)
    print(f"--- Chatting with {agent_name} ---")
    async for chunk in runner.run_async(user_input):
        print(chunk, end="")
    print("\n---------------------------------")

In [ ]:
# Test Scenario 1: Learning
await run_agent("MENTOR", "Explain recursion to me")

In [ ]:
# Test Scenario 2: Getting a Task
await run_agent("MANAGER", "I'm ready for a task on recursion")